In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.2MB/s eta 0:04:22tcmalloc: large alloc 1147494400 bytes == 0x55becb0ce000 @  0x7f7dae9b5615 0x55be929da06c 0x55be92ab9eba 0x55be929dce8d 0x55be92ace99d 0x55be92a50fe9 0x55be92a4bb0e 0x55be929de77a 0x55be92a50e50 0x55be92a4bb0e 0x55be929de77a 0x55be92a4d86a 0x55be92acf7c6 0x55be92a4cee2 0x55be92acf7c6 0x55be92a4cee2 0x55be92acf7c6 0x55be92a4cee2 0x55be92acf7c6 0x55be92b51431 0x55be92ab2049 0x55be92a1cc84 0x55be929dd8e9 0x55be92a51ade 0x55be929de69a 0x55be92a4ca45 0x55be92a4be0d 0x55be929de77a 0x55be92a4ca45 0x55be929de69a 0x55be92a4ca45
     |█████████████████████████████▏  | 1055.7MB 1.2MB/s eta 0:01:25tcmalloc: large alloc 1434370048 bytes == 0x55bf0f724000 @  0x7f7dae9b5615 0x55be929da06c 0x55be92ab9eba 0x55be929dce8d 0x55be92ace99d 0x55be92a50fe9 0x55be92a4bb0e 0x55be929de77a 0x55be92a50e50 0x55be92a4bb0e 0x55be929de77a 0x55be92a4d86a 0x55be92acf7c6 0x55be92a4cee2 0

In [5]:
!pip install --upgrade gensim mlflow tqdm

     |████████████████████████████████| 24.2MB 1.3MB/s 
     |████████████████████████████████| 14.2MB 245kB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 163kB 51.9MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 153kB 60.4MB/s 
     |████████████████████████████████| 1.1MB 42.0MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=13d0e4c27ba0d1684eeede8e05213cfffb08626a4daccaaf9d4db6d839e6a482
  Stored in directory: /root/.cache/pip/wheels/84/07/f7/12f7370ca47a66030c2edeedcc23dec26ea0ac22dcb4c4a0f3
  Created wheel for databricks-cli: filename=databricks_cli-0.14.3-cp37-none-any.whl size=100557 sha256=ef6493da4aa117d1718d181b9

In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [7]:
%%bash

mkdir -p data
#curl -L ./content/drive/My Drive/Colab Notebooks/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
#tar jxvf c -C ./data/
curl -L http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./data/SBW-vectors-300-min5.txt.bz2
bunzip2 ./data/SBW-vectors-300-min5.txt.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   259  100   259    0     0    316      0 --:--:-- --:--:-- --:--:--   315
100  780M  100  780M    0     0  15.9M      0  0:00:48  0:00:48 --:--:-- 18.7M
bunzip2: Output file ./data/SBW-vectors-300-min5.txt already exists.


In [8]:
%%bash

tar jxvf /content/drive/MyDrive/meli-challenge-2019.tar.bz2

tar (child): /content/drive/MyDrive/meli-challenge-2019.tar.bz2: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


## Librerías

In [9]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import IterableDataset, DataLoader
from tqdm.notebook import tqdm, trange

## Dataset

Nos basaremos en el dataset que creamos para el [notebook 2](./2_datasets.ipynb), con la diferencia de que pasaremos el dataframe de manera directa (esto es para poder hacer split en train/test).

In [10]:
!zcat ./meli-challenge-2019/spanish.train.csv.gz | wc -l

6119101


In [11]:
import numpy as np

train_index = np.random.rand(6119101) <= 0.8

with gzip.open("./meli-challenge-2019/spanish.train.csv.gz", "rt") as spanish_corpus,\
    gzip.open("./meli-challenge-2019/spanish-train.csv.gz", "wt") as spanish_train,\
    gzip.open("./meli-challenge-2019/spanish-validation.csv.gz", "wt") as spanish_validation:
  for idx, line in enumerate(tqdm(spanish_corpus, total=611910)):
    if train_index[idx]:
      spanish_train.write(line)
    else:
      spanish_validation.write(line)

  0%|          | 0/611910 [00:00<?, ?it/s]

In [12]:
!zcat ./meli-challenge-2019/spanish-train.csv.gz | wc -l

4895584


In [13]:
!zcat ./meli-challenge-2019/spanish-validation.csv.gz | wc -l

1223517


In [14]:
#!zcat ./meli-challenge-2019/spanish-test.csv.gz | wc -l

In [15]:
!zcat ./meli-challenge-2019/spanish-validation.csv.gz | head

spanish,reliable,Fifa18 Ps4 Disco Fisico,VIDEO_GAMES
spanish,reliable,Botines Futbol adidas Messi 15.4 Cesped Hombre,FOOTBALL_SHOES
spanish,reliable,Chops Sublimados - Nagual,DRINKING_GLASSES
spanish,reliable,Singer Hd205c Semi Industrial Motor 1100ppm Cabezal,SEWING_MACHINES
spanish,reliable,Ventilador De Techo Centrex Madera 4 Palas Con 2 Luces,FANS
spanish,reliable,Mochila Las Oreiro Original!súper Oferta!envio Gratis!,BACKPACKS
spanish,reliable,Rolling Stone 91 Rolling Stones,MAGAZINES
spanish,reliable,Memoria Pc 8gb Kingston Hyperx Fury Ddr3 1600mhz Azul,RAM_MEMORY_MODULES
spanish,reliable,Bravecto 4 A 10 Kg,FLEA_AND_TICK_TREATMENTS
spanish,reliable,Molde Silicona Abecedario Banderines Porcelana Fria Tort,KITCHEN_MOLDS


In [16]:
import csv
import random

class MeliChallengeDataset(IterableDataset):
    def __init__(self,
                 dataset_path,
                 dataset_size,
                 random_buffer_size=2048,
                 transform=None):
        assert random_buffer_size > 0
        self.dataset_path = dataset_path
        self.dataset_size = dataset_size
        self.random_buffer_size = random_buffer_size
        self.transform = transform
    
    def __len__(self):
        return self.dataset_size

    def __iter__(self):
        try:
            with gzip.open(self.dataset_path, "rt") as dataset:
                shuffle_buffer = []
                csv_reader = csv.reader(dataset)
                next(csv_reader)

                for line in csv_reader:
                    _, label_quality, title, category = line

                    if self.random_buffer_size == 1:
                        item = {
                            "title": f"{label_quality} {title}",
                            "category": category
                        }
                        if self.transform:
                            item = self.transform(item)
                        yield item
                    else:
                        shuffle_buffer.append({
                            "title": f"{label_quality} {title}",
                            "category": category
                        })

                        if len(shuffle_buffer) == self.random_buffer_size:
                            random.shuffle(shuffle_buffer)
                            for item in shuffle_buffer:
                                if self.transform:
                                    item = self.transform(item)
                                yield item
                            shuffle_buffer = []

            if len(shuffle_buffer) > 0:
                random.shuffle(shuffle_buffer)
                for item in shuffle_buffer:
                    if self.transform:
                        item = self.transform(item)
                    yield item
        except GeneratorExit:
            return

## Preprocesamiento

En este caso vamos a utilizar un sólo módulo para transformar los datos de IMDB. Este se encargará de preprocesar el texto (i.e. normalizarlo) y transformará las palabras en índices de un diccionario para luego poder pasar una secuencia de palabras para buscar en la matriz de embeddings y así permitir mayor manipulación de los embeddings (en lugar de utilizar embeddings fijos).

In [17]:
class RawDataProcessor:
    def __init__(self, 
                 dataset_path,
                 dataset_size,
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary()
        labels = set()

        with gzip.open(dataset_path, "rt") as dataset:
            csv_reader = csv.reader(dataset)
            if ignore_header:
              next(csv_reader)
            for line in tqdm(csv_reader, total=dataset_size):
                _, label_quality, title, category = line
                self.dictionary.add_documents(
                    [self._preprocess_string(f"{label_quality} {title}")]
                )
                labels.add(category)
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(labels)
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target, start=1)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx.get(target, 0)
    
    def __call__(self, item):
        if isinstance(item, list):
          data = [self.encode_data(i["title"]) for i in item]
          target = [self.encode_target(i["category"]) for i in item]
        else:
          data = self.encode_data(item["title"])
          target = self.encode_target(item["category"])
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos

En esta ocasión, leeremos los datos de IMDB y lo dividiremos en subconjuntos de entrenamiento y evaluación.

In [18]:
preprocess = RawDataProcessor(
    "./meli-challenge-2019/spanish-train.csv.gz",
    4895584
)

  0%|          | 0/4895584 [00:00<?, ?it/s]

In [19]:
train_dataset = MeliChallengeDataset(
    "./meli-challenge-2019/spanish-train.csv.gz",
    4895584,
    random_buffer_size=2048,
    transform = preprocess
)
validation_dataset = MeliChallengeDataset(
    "./meli-challenge-2019/spanish-validation.csv.gz",
    1223517,
    random_buffer_size=1,
    transform=preprocess
)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(validation_dataset)} validation elements")
print(f"Sample train element:\n{next(iter(train_dataset))}")

Datasets loaded with 4895584 training elements and 1223517 validation elements
Sample train element:
{'data': [4, 369, 402, 619, 618, 616, 371, 617], 'target': 295}


## Collation function

Como en este caso trabajamos con secuencias de palabras (representadas por sus índices en un vocabulario), cuando queremos buscar un *batch* de datos, el `DataLoader` de PyTorch espera que los datos del *batch* tengan la misma dimensión (para poder llevarlos todos a un tensor de dimensión fija). Esto lo podemos lograr mediante el parámetro de `collate_fn`. En particular, esta función se encarga de tomar varios elementos de un `Dataset` y combinarlos de manera que puedan ser devueltos como un tensor de PyTorch. Muchas veces la `collate_fn` que viene por defecto en `DataLoader` sirve (como se vio en el notbook 2), pero este no es el caso. Se define un módulo `PadSequences` que toma un valor mínimo, opcionalmente un valor máximo y un valor de relleno (*pad*) y dada una lista de secuencias, devuelve un tensor con *padding* sobre dichas secuencias.

In [21]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.LongTensor(target)
        }

## DataLoaders

Ya habiendo definido nuestros conjuntos de datos y nuestra `collation_fn`, podemos definir nuestros `DataLoader`s, uno para entrenamiento y otro para evaluación. Ver que la diferencia fundamental está en `shuffle`, no queremos mezclar los valores de evaluación cada vez que evaluamos porque al evaluar mediante *mini-batchs* nos puede generar inconsistencias.

In [22]:
EPOCHS = 2
FILTERS_COUNT = 100
FILTERS_LENGTH = [2, 3, 4]
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False,
                          collate_fn=pad_sequences, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
 #                              collate_fn=pad_sequences, drop_last=False)

In [23]:
next(iter(train_loader))

{'data': tensor([[    4,  1251,   145,  2263,  2704,  2262,  2705,  2704,     0,     0,
              0],
         [    4,   386,   385,   581,  3320,  3321,  1547,  2942,     0,     0,
              0],
         [    4,  2770,  1631,  2771,  2587,  1173,     1,     0,     0,     0,
              0],
         [    4,   663,  2263,  2265,  2262,   502,  1309,  2264,     0,     0,
              0],
         [    4,   875,  2200,  2199,  2198,  2196,  2197,  2201,  2195,     0,
              0],
         [    4,   122,   144,  1016,     0,     0,     0,     0,     0,     0,
              0],
         [    4,  1612,  1694,  1696,  2966,  1697,  3701,     0,     0,     0,
              0],
         [    4,   214,   566,    52,  3020,  3143,     0,     0,     0,     0,
              0],
         [    4,    30,    28,    32,    29,    33,    31,     0,     0,     0,
              0],
         [    4,  3374,  3378,  3377,   197,  3376,  3375,  2180,     0,     0,
              0],
         [  

## El modelo de clasificación

Para clasificación utilizaremos un perceptrón multicapa de dos capas ocultas. Claramente este modelo es naive y prácticamente todo lo que está *hardcodeado* (e.g. los tamaños de las capas o la cantidad de capas) podría ser parte de los parámetros del modelo. En particular, tenemos la capa de `Embeddings` que es rellenada con los valores de embeddings preentrenados (los de Glove en este caso).

In [24]:
!head ./data/SBW-vectors-300-min5.txt 

1000653 300
de -0.029648 0.011336 0.019949 -0.088832 -0.025225 0.056844 0.025473 0.014068 0.163694 -0.067154 0.014738 0.027134 0.066443 -0.044846 -0.044987 -0.040898 0.030311 0.034196 -0.049240 0.008537 -0.068091 -0.087938 0.035300 0.149385 -0.012350 0.012613 0.029350 0.069596 0.039111 0.057652 0.069954 -0.066217 -0.041784 0.028623 0.026772 -0.066392 0.002953 -0.012188 -0.030363 0.040222 0.034858 0.027469 -0.029034 -0.048748 -0.038582 -0.051553 -0.033501 -0.019008 0.003043 0.110712 -0.025096 0.111082 0.035244 0.114207 0.010195 0.051511 -0.040649 -0.113944 0.044873 0.052011 0.067360 0.049054 -0.127085 -0.031846 0.032848 0.040825 -0.084873 0.059801 -0.067424 0.016531 -0.084565 0.057024 0.083288 -0.010136 -0.048508 0.051757 0.046664 0.018102 -0.052320 -0.000765 0.053662 -0.009967 0.082858 0.009068 0.054575 -0.003466 -0.023376 0.023069 0.088513 0.018504 -0.039503 -0.032980 -0.002139 0.000010 -0.107627 0.007699 0.046351 -0.003062 0.030500 0.113650 0.032536 -0.097301 -0.013734 0.098345 0.080

In [25]:
class MeliChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 n_labels,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with open(pretrained_embeddings_path, "rt") as fh:
            next(fh)
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.convs = []
        for filter_lenght in FILTERS_LENGTH:
            self.convs.append(
                nn.Conv1d(vector_size, FILTERS_COUNT, filter_lenght)
            )
        self.convs = nn.ModuleList(self.convs)
        self.fc = nn.Linear(FILTERS_COUNT * len(FILTERS_LENGTH), 128)
        self.output = nn.Linear(128, n_labels)
        self.vector_size = vector_size
    #Acá hay que reestructurar los datos :) 
    @staticmethod    
    def conv_global_max_pool(x, conv):
        return F.relu(conv(x).transpose(1, 2).max(1)[0])
    
    def forward(self, x):
        x = self.embeddings(x).transpose(1, 2)  # Conv1d takes (batch, channel, seq_len)
        x = [self.conv_global_max_pool(x, conv) for conv in self.convs]
        x = torch.cat(x, dim=1)
        x = F.relu(self.fc(x))
        x = self.output(x)
        return x
        

In [26]:
model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                preprocess.dictionary, 
                                len(preprocess.target_to_idx) + 1,
                                300, True)

In [27]:
print(model)

MeliChallengeClassifier(
  (embeddings): Embedding(50002, 300, padding_idx=0)
  (convs): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
  )
  (fc): Linear(in_features=300, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=633, bias=True)
)


In [28]:
device = torch.device("cuda")
model = model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
for epoch in trange(EPOCHS):
  model.train()
  running_loss = []
  for idx, batch in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    data = batch["data"].to(device)
    target = batch["target"].to(device)
    output = model(data)
    loss_value = loss(output, target)
    loss_value.backward()
    optimizer.step()
    running_loss.append(loss_value.item())
  print("train_loss", sum(running_loss) / len(running_loss), epoch)  

  model.eval()
  running_loss = []
  targets = []
  predictions = []
  for batch in tqdm(validation_loader):
    data = batch["data"].to(device)
    target = batch["target"].to(device)
    output = model(data)
    running_loss.append(
      loss(output, target).item()
    )
    targets.extend(target.squeeze().cpu().detach().numpy())
    predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
  print("validation_loss", sum(running_loss) / len(running_loss), epoch)
  print("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/76494 [00:00<?, ?it/s]

train_loss 1.5652282638826673 0


  0%|          | 0/19118 [00:00<?, ?it/s]

validation_loss 12.44574382768065 0
validation_bacc 0.24393837153840406 0


  0%|          | 0/76494 [00:00<?, ?it/s]

train_loss 1.3100939850196354 1


  0%|          | 0/19118 [00:00<?, ?it/s]

validation_loss 16.04586812692783 1
validation_bacc 0.25046699266706873 1


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


## Experimento de MLflow

Por último, ya tenemos todos los bloques para construir nuestro experimento de MLflow. Anotamos un par de parámetros  (estos pueden ser todos los que se consideren necesarios) y lanzamos a correr nuestro experimento. Cada vez que finaliza un epoch guardamos algunas métricas. Al finalizar todos los epochs corremos algunas métricas extras de evaluación y guardamos algunos datos extra que nos servirán para calcular otras métricas a futuro.

In [ ]:
mlflow.set_experiment("a_naive_experiment")

with mlflow.start_run():
    mlflow.log_param("model_name", "mlp")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 50,
        "hidden1_size": 128,
        "hidden2_size": 128
    })
    model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                    preprocess.dictionary, 
                                    len(preprocess.target_to_idx) + 1,
                                    300, True)
    device = torch.device("cuda")
    model = model.to(device)
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(3):
        model.train()
        running_loss = 0
        steps=0
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            loss_value = loss(output, target)
            loss_value.backward()
            optimizer.step()
            running_loss +=loss_value.item()
            steps += 1       
        mlflow.log_metric("train loss", running_loss / steps, epoch)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(validation_loader):
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().cpu().detach().numpy())
        #mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)      
        

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/76494 [00:00<?, ?it/s]

  0%|          | 0/19118 [00:00<?, ?it/s]

In [ ]:
!tar zcvf mlruns.tar.gz mlruns

In [ ]:
%%bash

# Una vez que lo descargan, lo descomprimen en su máquina con el siguiente comando
tar xvf mlruns.tar.gz

# Luego ejecutan en el lugar que lo descomprimieron lo siguiente
mlflow ui